In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
from tqdm import tqdm

# --- スター数を数値に変換 ---
def parse_star_count(star_text):
    star_text = star_text.strip().lower().replace(",", "")
    if "k" in star_text:
        return int(float(star_text.replace("k", "")) * 1000)
    elif "m" in star_text:
        return int(float(star_text.replace("m", "")) * 1000000)
    else:
        try:
            return int(star_text)
        except ValueError:
            return 0

# --- DB設定 ---
db_name = 'google_repos.db'

# --- スクレイピング設定 ---
headers = {"User-Agent": "Mozilla/5.0"}
base_url = "https://github.com/orgs/google/repositories?page={}"

total_repos = 0
page = 1

try:
    with sqlite3.connect(db_name) as conn:
        cur = conn.cursor()
        cur.execute('''
        CREATE TABLE IF NOT EXISTS repositories (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT UNIQUE,
            language TEXT,
            stars INTEGER
        );
        ''')

        with tqdm(total=0, unit="repo", dynamic_ncols=True,
                  bar_format="{desc} [{elapsed}<{remaining}, {rate_fmt}]") as pbar:
            while True:
                url = base_url.format(page)

                # ✅ ページ取得前に1秒待機
                time.sleep(1)

                res = requests.get(url, headers=headers, timeout=10)
                res.raise_for_status()
                soup = BeautifulSoup(res.text, "html.parser")
                repo_blocks = soup.find_all('li', class_="ListItem-module__listItem--k4eMk")

                if not repo_blocks:
                    print(f"ページ {page} にリポジトリが見つかりません。終了します。")
                    break

                pbar.total += len(repo_blocks)
                pbar.refresh()

                batch_data = []

                for repo_block in repo_blocks:
                    try:
                        # --- コード2の抽出ロジックを追加 ---
                        # リポジトリ名
                        a_tag = repo_block.find('a', class_="Title-module__anchor--GmXUE Title-module__inline--oM0P7")
                        repo_name = a_tag.get_text(strip=True) if a_tag else None
                        if not repo_name:
                            pbar.update(1)
                            continue

                        # 言語
                        language_tag = repo_block.find('span', class_='ReposListItem-module__Text_4--mkG7R')
                        language = language_tag.get_text(strip=True) if language_tag else "Unknown"

                        # スター数
                        star_tag = repo_block.find(lambda tag: tag.name == "a" and tag.get("aria-label") and "stars" in tag.get("aria-label"))
                        stars = parse_star_count(star_tag.get_text(strip=True)) if star_tag else 0

                        batch_data.append((repo_name, language, stars))

                    except Exception as e:
                        print(f"データ処理エラー（ページ {page}）: {e}")
                    finally:
                        pbar.update(1)
                        pbar.set_description(
                            f"ページ {page} | 取得済み {total_repos} 件 | 暫定合計件数 {pbar.total} 件"
                        )

                if batch_data:
                    cur.executemany("INSERT OR IGNORE INTO repositories (name, language, stars) VALUES (?, ?, ?)", batch_data)
                    conn.commit()

                total_repos += len(batch_data)
                page += 1

        print("\n✅ 全ページ取得完了！")
        print(f"総件数：{total_repos} 件")

        # --- データ確認 ---
        try:
            print("\n--- DB内容 ---")
            for row in cur.execute("SELECT * FROM repositories"):
                print(row)
        except sqlite3.Error as e:
            print(f"DB読み込みエラー: {e}")

except Exception as e:
    print(f"予期せぬエラー: {e}")

ページ 94 | 取得済み 2790 件 | 暫定合計件数 2811 件:  [02:57<00:00, 15.85repo/s]

ページ 95 にリポジトリが見つかりません。終了します。

✅ 全ページ取得完了！
総件数：2811 件

--- DB内容 ---
(1, 'security-research', 'C', 4300)
(2, 'syzkaller', 'Go', 5900)
(3, 'adk-java', 'Java', 946)
(4, 'perfetto', 'C++', 5000)
(5, 'dawn', 'C++', 770)
(6, 'gemma.cpp', 'C++', 6600)
(7, 'yggdrasil-decision-forests', 'C++', 620)
(8, 'cel-java', 'Java', 227)
(9, 'timesketch', 'Python', 3200)
(10, 'device-infra', 'Java', 58)
(11, 'error-prone', 'Java', 7100)
(12, 'fonts', 'HTML', 19000)
(13, 'XNNPACK', 'C', 2200)
(14, 'osv-scalibr', 'Go', 534)
(15, 'oss-fuzz', 'Shell', 12000)
(16, 'nomulus', 'Java', 1800)
(17, 'or-tools', 'C++', 13000)
(18, 'ground-platform', 'TypeScript', 226)
(19, 'koladata', 'C++', 27)
(20, 'built_value.dart', 'Dart', 882)
(21, 'brax', 'Jupyter Notebook', 2900)
(22, 'gvisor', 'Go', 17000)
(23, 'angle', 'C++', 3800)
(24, 'seesaw', 'Go', 5700)
(25, 'oboe', 'C++', 4000)
(26, 'orbax', 'Python', 454)
(27, 'boringssl', 'C++', 2100)
(28, 'xls', 'C++', 1400)
(29, 'adk-go', 'Go', 4300)
(30, 'tunix', 'Python', 1900)
(